In [ ]:
# EOQ PROBILITY MODEL Q CONTINOUS REVIEW

import numpy as np
from scipy.stats import norm

def calculate_inventory(D, A, h, Cu, l, S_d, S_l, tol=0.01, max_iter=100):
    """
    Menghitung parameter model persediaan probabilistik menggunakan iterasi hingga r* konvergen.
    """
    # Menghitung q_0^* awal (EOQ)
    q_0_star = np.sqrt((2 * A * D) / h)
    q_02_star = q_0_star  # Inisialisasi q_02^*

    iterasi = 0
    r_1_star = 0  # Nilai awal reorder point
    SS_final = 0  # Inisialisasi safety stock

    while iterasi < max_iter:
        # 1. Menghitung kembali α berdasarkan q_02^*
        alpha = 1 - (h * q_02_star) / (Cu * D)

        # 2. Menghitung kembali Z_α
        Z_alpha = abs(norm.ppf(1 - alpha))

        # 3. Menghitung standar deviasi permintaan selama lead time (S_dl)
        S_dl = np.sqrt((D / 12) ** 2 * S_l ** 2 + (l * S_d ** 2))

        # 4. Menghitung fungsi distribusi normal f(Z_alpha) dan ψ(Z_alpha)
        f_Z_alpha = norm.pdf(Z_alpha)
        psi_Z_alpha = 1 - norm.cdf(Z_alpha)

        # 5. Menghitung N
        N = S_dl * (f_Z_alpha - Z_alpha * psi_Z_alpha)

        # 6. Menghitung q_02^* yang baru
        q_02_star_new = np.sqrt((2 * D * (A + Cu * N)) / h)

        # 7. Menghitung safety stock (SS)
        SS_final = Z_alpha * S_dl

        # 8. Menghitung reorder point r_2^*
        d = D / 12  # Permintaan rata-rata bulanan
        r_2_star = (d * l) + SS_final

        # 9. Periksa konvergensi
        if abs(r_2_star - r_1_star) < tol and abs(q_02_star_new - q_02_star) < tol:
            q_0_star = q_02_star_new  # Set q_0^* setelah iterasi konvergen
            N_final = N  # Simpan nilai N setelah iterasi konvergen
            break

        # 10. Perbarui nilai untuk iterasi selanjutnya
        q_02_star = q_02_star_new
        r_1_star = r_2_star
        iterasi += 1

    return q_0_star, r_2_star, SS_final, iterasi

# Parameter dari dokumen
D = 672  # Permintaan tahunan di 2025 hasil arima
A = 750000  # Biaya pemesanan
h = 550900  # Biaya penyimpanan
Cu = 230000  # Biaya kekurangan
l = 1.87  # Lead time rata-rata (bulan)
S_d = 4.58  # Standar deviasi permintaan bulanan
S_l = 1.74  # Standar deviasi lead time (bulan)

# Menjalankan perhitungan
q_0_star, r_star, SS_final, iterasi = calculate_inventory(D, A, h, Cu, l, S_d, S_l)

# Menampilkan hasil akhir
print(f"Ukuran lot pemesanan optimal (q_0^*): {q_0_star:.2f} unit")
print(f"Reorder point (r^*): {r_star:.2f} unit")
print(f"Safety Stock (SS): {SS_final:.2f} unit")
print(f"Jumlah iterasi: {iterasi}")


Ukuran lot pemesanan optimal (q_0^*): 147.90 unit
Reorder point (r^*): 111.36 unit
Safety Stock (SS): 6.64 unit
Jumlah iterasi: 32
